# NRGV Data Analyst Interview Project

## Data Munging_2

### Future Work - Ideas

1) Could test for count of underscores in file name to chack for additional encodings<br>
2) Using LAT/LONG and Date, determine sunset/sunrise; progamaticaly remove records when the sun is not out (from sunset to sunrise).<br>
2a) Eliminate these records during the injest stage -Document time-delta vs lines of code vs reduced records.<br>
3) Automate the detection of new state in dictionary.

### Objectives:

<ul>
    <li>File consolidation / Create a single file with values from all states</li>
    <li>Aggregrate Values </li>
</ul>

-----------------------------

### Libraries

In [11]:
import requests as req     # DOWNLOADING URL VIA URL
import zipfile             # DEALING WITH ZIP COMPRESSED FILE
from io import BytesIO     # READING FILE FROM BUFFER
import os                  # DIRECTORY CREATION
import pandas as pd        # PANDAS DATAFRAME
import datetime            # DATE AND TIME
from datetime import datetime
import numpy as np

### Variables

In [3]:
# PATH TO PLACE DATA
data_path = 'C://Users//bruce//Documents//Data//NRGV'

# SWSD_SUMMARY_DATA 
sumry_data_path = 'C://Users//bruce//Documents//Data//SWSD_SUMMARY_DATA'

# SWSD RECORDS REDUCTION
record_reduction_path_filename = sumry_data_path + "//Record_Reduction_Statistics.txt"

### READ EACH STATE_SWSD_SUMMARY_DATA_DAYTIME FILE AND APPEND IT INTO SINGLE FILE

In [20]:
file_cnt = 0

for smry_data_file_daytime in os.listdir(sumry_data_path):
       
    mod_smry_data_file_nme = smry_data_file_daytime[3:28]  
    
    if mod_smry_data_file_nme == 'SWSD_SUMMARY_DATA_DAYTIME':
        
        file_cnt += 1
        
        if file_cnt == 1:
            
            temp_data = pd.read_csv(sumry_data_path + "//"+ smry_data_file_daytime, header=0) #, parse_dates=['LocalTime']) ,index_col=0)
        
            #msg = "The length of the current temp_data field is: {:,}".format(len(temp_data))
            
            #print(msg)
            
            swsd_daytime_append = temp_data
        
        elif file_cnt > 1:
            
            temp_data = pd.read_csv(sumry_data_path + "//"+ smry_data_file_daytime, skiprows=0) #, parse_dates=['LocalTime']) ,index_col=0)
        
            #msg = "The length of the current temp_data field is: {:,}".format(len(temp_data))
            
            #print(msg)                
                
            swsd_daytime_append = swsd_daytime_append.append(temp_data)
        
        #print('The file_cnt is: {}'.format(file_cnt))
        #print("The length of the current swsd_daytime_append field is: {:,}".format(len(swsd_daytime_append)))
                  
# Remove column name 'A'
swsd_daytime_append = swsd_daytime_append.drop(['WeatherYear'], axis = 1)
            
# WRITE NEW ALL_STATES_SWSD_SUMMARY_DATA_DAYTIME.csv
swsd_daytime_append.to_csv(sumry_data_path + "//ALL_STATES_SWSD_SUMMARY_DATA_DAYTIME.csv", header=True, index=False)        
        

In [21]:
print("The swsd_daytime_append file contains {:,} records.".format(len(swsd_daytime_append)))

The swsd_daytime_append file contains 66,722,730 records.


In [147]:
# DEFINE COLUMNS THAT ARE BEING GROUPBY
grb_1 = swsd_daytime_append.groupby(['State','DataType','Latitude','Longitude','PVType','CapacityMW'])

# BASED UPON THE COLUMNS THAT ARE BEING GROUPBY, DEFINE THE INDIVIDUAL COLUMN AGGERGATIONS {'<column_1': [<agg_1>,<agg_2],
#                                                                                           '<column_2': [<agg_1>,<agg_3]}
# .reset_index() WILL REST INDEX AND COLUMN VAULES WILL REPEAT 

powerMW_agg = grb_1[['Power(MW)','TimeIntervalMin']].agg({'Power(MW)' : [np.sum, max, np.mean],\
                                                          'TimeIntervalMin': [np.sum]}).reset_index()

In [148]:
# CALCULATE NEW COLUMN - ['MeanEffectivness']
#powerMW_agg['MeanEffectivness'] = powerMW_agg['Power(MW)', 'mean'] / powerMW_agg['CapacityMW']

# SORT THE ['MeanEffectivness'] COLUMN BASED UPON DESCENDING VALUES
#powerMW_agg = powerMW_agg.sort_values(by=['MeanEffectivness'], ascending=False)

In [149]:
# WRITE NEW ALL_STATES_SWSD_SUMMARY_DATA_DAYTIME.csv
powerMW_agg.to_csv(sumry_data_path + "//ALL_STATES_SWSD_SUMMARY_DATA_DAYTIME_AGGREGATIONS.csv", header=True, index=False)

In [150]:
powerMW_agg.head()

State DataType Latitude Longitude PVType CapacityMW  Power(MW)        \
                                                             sum   max   
0    AL   Actual    30.45    -88.25    UPV         70  1200879.0  61.0   
1    AL   Actual    30.55    -88.25    DPV         38   630932.4  32.2   
2    AL   Actual    30.55    -88.15    DPV         38   636177.5  31.9   
3    AL   Actual    30.55    -87.75    DPV         36   597170.9  31.5   
4    AL   Actual    30.55    -87.55    UPV         80  1363291.2  74.6   

             TimeIntervalMin  
        mean             sum  
0  17.688599          339450  
1   9.293451          339450  
2   9.370710          339450  
3   8.796154          339450  
4  20.080884          339450

In [146]:
powerMW_agg[powerMW_agg['MeanEffectivness']==powerMW_agg['MeanEffectivness'].max()]

State DataType Latitude Longitude PVType CapacityMW Power(MW)        \
                                                               sum   max   
1335    CA       DA    32.65   -116.15    UPV         50  128360.4  45.0   

                TimeIntervalMin MeanEffectivness  
           mean             sum                   
1335  21.979521          350400          0.43959